In [740]:
import pandas as pd
import geopandas as gpd
import numpy as np
from housing_elements import utils, los_altos_permits, san_francisco_permits, san_jose_permits

In [755]:
from importlib import reload
reload(los_altos_permits)
reload(san_francisco_permits)
reload(utils)

<module 'housing_elements.utils' from '/Users/Salim/Desktop/housing-elements/housing_elements/utils.py'>

In [742]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [743]:
# Oakland and Palo Alto don't work right now (parsing certain columns as floats fails)
cities = ['San Francisco', 'Berkeley', 'Mountain View', 'San Jose']

In [744]:
df = gpd.read_file(
        "./data/raw_data/housing_sites/xn--Bay_Area_Housing_Opportunity_Sites_Inventory__20072023_-it38a.shp"
    )

In [745]:
all_cities = df.jurisdict.unique()

In [746]:
len(all_cities)

108

In [747]:
%%time
cities_with_sites = {}

for city in all_cities:
    try:
        with HiddenPrints():
            cities_with_sites[city] = utils.load_site_inventory(city)
            cities_with_sites[city].relcapcty.sum()
    except Exception:
        pass

CPU times: user 4min 45s, sys: 5.49 s, total: 4min 50s
Wall time: 4min 48s


In [748]:
len(cities_with_sites)

108

In [749]:
%%time
cities_with_permits = {}

for city in all_cities:
    try:
        with HiddenPrints():
            cities_with_permits[city] = utils.load_all_new_building_permits(city)
    except Exception:
        pass

CPU times: user 4min 20s, sys: 1.66 s, total: 4min 22s
Wall time: 4min 24s


In [750]:
len(cities_with_permits)

0

In [751]:
def get_results_for_city(city: str, merge_with_address: bool = False) -> pd.DataFrame:
    permits = utils.load_all_new_building_permits(city)
    sites = utils.load_site_inventory(city)
    
    return {
        'City': city,
        'Ratio of development on inventory sites': utils.calculate_total_units_permitted_over_he_capacity(sites, permits),
        'Mean overproduction': utils.calculate_underproduction_on_sites(sites, permits),
        'Inventory housing / all housing': utils.calculate_inventory_housing_over_all_housing(sites, permits),
        'P(dev) for nonvacant sites': utils.calculate_pdev_for_nonvacant_sites(sites, permits, merge_with_address),
        'P(dev) for vacant sites': utils.calculate_pdev_for_vacant_sites(sites, permits, merge_with_address),
        'P(dev) for inventory': utils.calculate_pdev_for_inventory(sites, permits, merge_with_address),
    }

In [454]:
cities = [city for city in all_cities if city in cities_with_permits and city in cities_with_sites]

In [455]:
len(cities)

96

In [632]:
%%time
with HiddenPrints():
    results_df = pd.DataFrame([get_results_for_city(city) for city in cities])

CPU times: user 8min 8s, sys: 7.29 s, total: 8min 15s
Wall time: 8min 13s


In [752]:
round(results_df, 2)

,City,Ratio of development on inventory sites,Mean overproduction,Inventory housing / all housing,P(dev) for nonvacant sites,P(dev) for vacant sites,P(dev) for inventory
0,Berkeley,0.35,3.22,0.04,0.01,0.03,0.02
1,Albany,0.63,NaN,0.00,0.00,0.00,0.00
2,Alameda,0.53,NaN,0.00,0.00,0.00,0.00
3,Livermore,0.58,1.00,0.24,0.07,0.17,0.12
4,Fremont,0.65,-19.39,0.15,0.07,0.05,0.07
...,...,...,...,...,...,...,...
91,Monte Sereno,1.42,NaN,0.00,0.00,NaN,0.00
92,Healdsburg,0.51,-7.50,0.03,0.08,0.08,0.08
93,Fairfield,0.32,-7.00,0.00,0.00,0.03,0.02
94,Sonoma,0.18,-35.00,0.03,0.00,0.20,0.04


In [44]:
%%time
results_geo_df = pd.DataFrame([get_results_for_city(city, merge_with_address=True) for city in cities])

/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (6158, 16)
Total units permitted: 26071
Total realistic capacity in inventory: 47209.0
Number of inventory sites developed: 0
Number of units permitted on inventory sites: 0
Total realistic capacity of inventory sites: 0.0
Units permitted on inventory sites: 0
Total units permitted: 26071


/Users/sidharthkapur/personal-workspace/housing-elements/housing_elements/utils.py:299: RuntimeWarning: invalid value encountered in double_scalars
  return (n_units - n_claimed) / len(inventory_sites_permitted)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (404, 18)
Total units permitted: 1871
Total realistic capacity in inventory: 5372.02
Number of inventory sites developed: 0
Number of units permitted on inventory sites: 0
Total realistic capacity of inventory sites: 0.0
Units permitted on inventory sites: 0
Total units permitted: 1871


/Users/sidharthkapur/personal-workspace/housing-elements/housing_elements/utils.py:299: RuntimeWarning: invalid value encountered in double_scalars
  return (n_units - n_claimed) / len(inventory_sites_permitted)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'num_vl' 'num_l' 'num_m'
 'num_am']
DF shape (93, 24)
Total units permitted: 3579.0
Total realistic capacity in inventory: 3216.0
Number of inventory sites developed: 3
Number of units permitted on inventory sites: 426.0
Total realistic capacity of inventory sites: 799.0
Units permitted on inventory sites: 426.0
Total units permitted: 3579.0


/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (599, 18)
Total units permitted: 16928.0
Total realistic capacity in inventory: 35536.43
Number of inventory sites developed: 64
Number of units permitted on inventory sites: 3473.0
Total realistic capacity of inventory sites: 3664.0
Units permitted on inventory sites: 3473.0
Total units permitted: 16928.0
CPU times: user 55.1 s, sys: 1.27 s, total: 56.3 s
Wall time: 57 s


In [45]:
results_geo_df

,City,Ratio of development on inventory sites,Mean overproduction,Inventory housing / all housing,P(dev) for nonvacant sites,P(dev) for vacant sites,P(dev) for inventory
0,San Francisco,0.552246,NaN,0.000000,0.074924,0.075974,0.075187
1,Berkeley,0.348286,NaN,0.000000,0.029940,0.033755,0.032178
2,Mountain View,1.112873,-124.333333,0.119028,0.186047,0.285714,0.193548
3,San Jose,0.476356,-2.984375,0.205163,0.236641,0.126214,0.198664


In [667]:
def get_ground_truth_results_for_city(city: str) -> pd.DataFrame:
    if city == 'San Jose':
        permits = san_jose_permits.load_all_permits()
    elif city == 'San Francisco':
        permits = san_francisco_permits.load_all_permits()
    elif city == 'Los Altos':
        permits = los_altos_permits.load_all_permits()
    else:
        raise ValueError(f"Ground truth data not available for {city}")
        
    permits = utils.load_all_new_building_permits(city)
    sites = utils.load_site_inventory(city)
    
    return {
        'City': city,
        'Ratio of development on inventory sites': utils.calculate_total_units_permitted_over_he_capacity(sites, permits),
        'Mean overproduction': utils.calculate_underproduction_on_sites(sites, permits),
        'Inventory housing / all housing': utils.calculate_inventory_housing_over_all_housing(sites, permits),
        'P(dev) for nonvacant sites': utils.calculate_pdev_for_nonvacant_sites(sites, permits),
        'P(dev) for vacant sites': utils.calculate_pdev_for_vacant_sites(sites, permits),
        'P(dev) for inventory': utils.calculate_pdev_for_inventory(sites, permits),
    }

In [672]:

permits = pd.read_csv("./data/raw_data/sf_permits.csv")
print(permits.columns)
date_cols = [c for c in permits.columns if 'Date' in c]
permits[date_cols] = permits[date_cols].apply(pd.to_datetime)

Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')


In [673]:
print(permits.columns)

Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')


In [756]:
%%time
ground_truth_cities = ['Los Altos', 'San Francisco', 'San Jose']
ground_truth_results_df = pd.DataFrame([get_ground_truth_results_for_city(city) for city in ground_truth_cities])

Dropping constant columns: ['county', 'num_vl', 'rhnayrs', 'rhnacyc', 'infcapcty', 'allowlow', 'jurisdict', 'num_m', 'num_am', 'num_l']
DF shape (57, 25)
Total units permitted: 532
Total realistic capacity in inventory: 257.5
Number of inventory sites developed: 9
Number of units permitted on inventory sites: 34
Total realistic capacity of inventory sites: 38.0
Units permitted on inventory sites: 34
Total units permitted: 532


/Users/Salim/opt/miniconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (18,22,32,35) have mixed types.Specify dtype option on import or set low_memory=False.
  
/Users/Salim/Desktop/housing-elements/housing_elements/san_francisco_permits.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ]


Dropping constant columns: ['county', 'num_vl', 'rhnayrs', 'rhnacyc', 'currunits', 'allowlow', 'jurisdict', 'num_m', 'num_am', 'num_l']
DF shape (6158, 25)
Total units permitted: 26071
Total realistic capacity in inventory: 47209.0
Number of inventory sites developed: 1
Number of units permitted on inventory sites: 6939
Total realistic capacity of inventory sites: 47209.0
Units permitted on inventory sites: 6939
Total units permitted: 26071
Dropping constant columns: ['county', 'num_vl', 'rhnayrs', 'rhnacyc', 'currunits', 'allowlow', 'jurisdict', 'num_m', 'num_am', 'num_l']
DF shape (599, 25)
Total units permitted: 16928.0
Total realistic capacity in inventory: 35536.43
Number of inventory sites developed: 67
Number of units permitted on inventory sites: 5011.0
Total realistic capacity of inventory sites: 6422.0
Units permitted on inventory sites: 5011.0
Total units permitted: 16928.0
CPU times: user 29.3 s, sys: 1.39 s, total: 30.7 s
Wall time: 1min 21s


In [684]:
ground_truth_results_df

,City,Ratio of development on inventory sites,Mean overproduction,Inventory housing / all housing,P(dev) for nonvacant sites,P(dev) for vacant sites,P(dev) for inventory
0,Los Altos,2.066019,-0.444444,0.06391,0.088235,0.26087,0.157895
